In [1]:
# импортирования библиотек
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# импорт датасета для обучения
data = pd.read_csv('train_data_final.csv')
data.head()

,Unnamed: 0,cost,level_1_1,level_1_2,level_1_3,level_2_1,level_2_2,region,title_vector_0,title_vector_1,...,85,86,87,88,89,90,price_sim,code_sim,title_sim,region_sim
0,0,1.76,0.0,0.0,1.0,0.0,1.0,64.0,-0.060832,-0.092878,...,0.0,0.0,0.0,0.000000,0.015152,0.0,1.485369e+06,1,0.883365,0.295455
1,1,1.76,0.0,0.0,1.0,0.0,1.0,64.0,-0.060832,-0.092878,...,0.0,0.0,0.0,0.000000,0.000000,0.0,4.367639e+04,1,0.846847,0.181818
2,2,1.76,0.0,0.0,1.0,0.0,1.0,64.0,-0.060832,-0.092878,...,0.0,0.0,0.0,0.000000,0.000000,0.0,4.224720e+06,1,0.740653,0.857143
3,3,1.76,0.0,0.0,1.0,0.0,1.0,64.0,-0.060832,-0.092878,...,0.0,0.0,0.0,0.000000,0.000000,0.0,1.850338e+06,1,0.747043,0.166667
4,4,1.76,0.0,0.0,1.0,0.0,1.0,64.0,-0.060832,-0.092878,...,0.0,0.0,0.0,0.010791,0.017986,0.0,4.773386e+05,1,0.870361,0.010791


In [3]:
data.drop('Unnamed: 0',axis = 1, inplace=True)

In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def interact_features(df, supplier):
    price_diff = abs(df['cost'] - supplier['supplier_cost'])
    level = str(int(df['level_1_1'])) + str(int(df['level_1_2'])) + str(int(df['level_1_3'])) + str(int(df['level_2_1'])) + str(int(df['level_2_2']))
    okpd = 1 if supplier[f"level_{level}"] > 0 else 0
    
    df_text = []
    supplier_text = []
    for i in range(20):
        df_text.append(df[f'title_vector_{i}'])
        supplier_text.append(supplier[f'supplier_title_vector_{i}'])
    desc_cosine_sim = cosine_similarity(np.array(df_text).reshape(1, -1), np.array(supplier_text).reshape(1, -1))[0][0]
    region = supplier[int(df['region'])]
    return [price_diff, okpd, desc_cosine_sim, region]

In [5]:
# чтения данных поставщика и удаления некоторых столбцов
supplier_df = pd.read_csv('supplier_data.csv')
supplier_df.drop('Unnamed: 0',axis = 1, inplace=True)
supplier_df.drop('level',axis = 1, inplace=True)
supplier_df

,supplier_id,supplier_title_vector_0,supplier_title_vector_1,supplier_title_vector_2,supplier_title_vector_3,supplier_title_vector_4,supplier_title_vector_5,supplier_title_vector_6,supplier_title_vector_7,supplier_title_vector_8,...,81,82,83,84,85,86,87,88,89,90
0,0,0.157682,0.080696,0.148246,0.115897,0.045616,-0.118755,-0.230366,0.241533,-0.239231,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
1,1,0.186906,0.029202,0.113319,0.054532,0.003567,-0.186207,-0.202754,0.242324,-0.264451,...,0.000000,0.000000,0.037037,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
2,2,0.191807,0.128104,0.238278,0.283948,0.031778,-0.012969,-0.284921,0.379179,-0.138028,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
3,3,-0.037172,0.149822,0.140487,0.179297,-0.001293,0.007779,-0.319230,0.289215,-0.157917,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
4,4,0.002321,-0.157497,0.352861,0.266140,-0.096408,-0.101884,-0.175416,0.250061,-0.073083,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,0.131686,0.096827,0.225170,0.056088,0.039205,-0.096672,-0.170438,0.175755,-0.145642,...,0.025000,0.050000,0.000000,0.0,0.0,0.0,0.05,0.0,0.025000,0.000000
616,616,0.067635,-0.222361,0.271203,0.063264,-0.209116,-0.117527,-0.297581,0.358772,0.012208,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.083333,0.000000
617,617,0.106666,0.024665,0.246442,0.023610,0.046229,-0.145921,-0.233765,0.248996,-0.164641,...,0.785714,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.142857
618,618,0.055878,0.165271,0.161804,0.095603,0.027726,0.013171,-0.320960,0.272917,-0.196184,...,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000


In [6]:
# сохранение в список поставщиков у которых только один тендер
res = data['tender_id'].value_counts()
tenders = list(res[res == 1].index)

In [7]:
# нахождения похожих по тендеру поставщиков
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
arr = []
interact_value1 = []
interact_value2 = []
interact_value3 = []
interact_value4 = []
x = 0
for tender_id in tenders:
    supplier = data[data['tender_id'] == tender_id]['supplier_id'].values[0]
    df_row = data[data['tender_id'] == tender_id].iloc[:,0:29]
    main_sup1 = supplier_df[supplier_df['supplier_id'] == supplier].rename({"supplier_id":"supplier"}, axis = 1)
    main_sup1.index = range(df_row.index[0],df_row.index[0]+1)
    inter = interact_features(df_row.iloc[0], main_sup1.iloc[0])
    interact_value1.append(inter[0])
    interact_value2.append(inter[1])
    interact_value3.append(inter[2])
    interact_value4.append(inter[3])
    new_row = pd.concat([df_row, main_sup1], axis = 1)
    arr.append(list(new_row.values[0]))
    
    
    for supplier_id in supplier_df['supplier_id']:
        if supplier_id != supplier:
            supplier_row = supplier_df[supplier_df['supplier_id']==supplier_id].rename({"supplier_id":"supplier"}, axis = 1)
            supplier_row.index = range(df_row.index[0],df_row.index[0]+1)
            inter = interact_features(df_row.iloc[0], supplier_row.iloc[0])
            if inter[1] == 1 and inter[2]>=0.30 and inter[3] > 0:
                interact_value1.append(inter[0])
                interact_value2.append(inter[1])
                interact_value3.append(inter[2])
                interact_value4.append(inter[3])
                new_row = pd.concat([df_row, supplier_row], axis = 1)
                arr.append(list(new_row.values[0]))

In [8]:
columns = list(data.iloc[0,0:152].index)

In [9]:
# создания датасета 
prep_df = pd.DataFrame(arr, columns = columns)
prep_df['price_sim'] = interact_value1
prep_df['code_sim'] = interact_value2
prep_df['title_sim'] = interact_value3
prep_df['region_sim'] = interact_value4

In [10]:
# сохранение в список все еще одиноких по тендеру поставщиков
res = prep_df['tender_id'].value_counts()
tenders = res[res == 1]

In [11]:
# соединение данных в один датасет
data = pd.concat([data, prep_df], axis = 0)

In [12]:
# удаление тендеров, у которых только один поставщик
data = data[~data['tender_id'].isin(tenders)]

In [13]:
# размер датасета
data.shape

(674553, 156)

In [14]:
len(tenders)

687

In [15]:
# добавления столбца метки для ранжирования
y_label = []
for i, j in zip(data['supplier_id'], data['supplier']):
    if i==j:
        y_label.append(1)
    else:
        y_label.append(0)

In [16]:

df = data.drop('supplier_id', axis = 1)
df['y'] = y_label
features = [col for col in df.columns if col not in ['tender_id','y']]
X = df[features]
y = df['y']
groups = df.groupby("tender_id").size().tolist()

# 2. Разделение данных по tender_id
unique_ids = df["tender_id"].unique()
train_ids, val_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

train_df = df[df["tender_id"].isin(train_ids)]
val_df = df[df["tender_id"].isin(val_ids)]

X_train = train_df[features]
y_train = train_df["y"]
group_train = train_df.groupby("tender_id").size().tolist()

X_val = val_df[features]
y_val = val_df["y"]
group_val = val_df.groupby("tender_id").size().tolist()

# 3. LightGBM Dataset
train_set = lgb.Dataset(X_train, label=y_train, group=group_train)
val_set = lgb.Dataset(X_val, label=y_val, group=group_val)

# 4. Обучение модели
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [1, 3, 5],
    "learning_rate": 0.05,
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "verbose": -1,
}

model = lgb.train(
    params,
    train_set,
    valid_sets=[train_set, val_set],
    valid_names=["train", "valid"],
    num_boost_round=100,
)

Средний NDCG: 0.65


In [25]:
# 5. Оценка точности
val_preds = model.predict(X_val)

# NDCG по группам
ndcg_scores = []
for tender_id, group in val_df.groupby("tender_id"):
    true_relevance = group["y"].values.reshape(1, -1)
    pred_scores = model.predict(group[features]).reshape(1, -1)
    score = ndcg_score(true_relevance, pred_scores)
    ndcg_scores.append(score)

print(f"Средний NDCG: {sum(ndcg_scores)/len(ndcg_scores):.2f}")

Средний NDCG: 0.65


In [17]:
# Предсказания
val_df["pred"] = model.predict(X_val)


val_result = val_df[["tender_id", "y", "pred"]].copy()

# Сортировка по tender_id и score
top_k = 5
top_k_preds = (
    val_result
    .sort_values(["tender_id", "pred"], ascending=[True, False])
    .groupby("tender_id")
    .head(top_k)
)

# Группируем по tender_id: проверяем, есть ли y=1 в топ-5
hit_per_group = top_k_preds.groupby("tender_id")["y"].max()

# Итоговая метрика
hit_rate_at_5 = hit_per_group.mean()

# точность того, что поставщик  попадет в топ 5 рекомендованных поставщиков
print(f"hit rate @5: {hit_rate_at_5:.2f}")

hit rate @5: 0.73


C:\Users\abuzu\AppData\Local\Temp\ipykernel_2324\3477190103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df["pred"] = model.predict(X_val)


In [18]:
# Предсказания
val_df["pred"] = model.predict(X_val)

val_result = val_df[["tender_id", "y", "pred"]].copy()

# Сортировка по tender_id и score
top_k = 10
top_k_preds = (
    val_result
    .sort_values(["tender_id", "pred"], ascending=[True, False])
    .groupby("tender_id")
    .head(top_k)
)

# Группируем по tender_id: проверяем, есть ли y=1 в топ-10
hit_per_group = top_k_preds.groupby("tender_id")["y"].max()

# Итоговая метрика
hit_rate_at_10 = hit_per_group.mean()

# точность того, что поставщик  попадет в топ 10 рекомендованных поставщиков
print(f"hit rate @10: {hit_rate_at_10:.2f}")

hit rate @10: 0.85


C:\Users\abuzu\AppData\Local\Temp\ipykernel_2324\3975009416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df["pred"] = model.predict(X_val)


In [26]:
# Получаем предсказания для валидационного набора
val_preds_scores = model.predict(X_val)

# Добавляем предсказанные скоры к валидационному датафрейму для удобства
val_df['pred_score'] = val_preds_scores

k = 5 
precisions_at_k = []
recalls_at_k = []
f1_scores_at_k = []
average_precisions_at_k = [] # Для MAP@k

print(f"\nРасчет метрик для K={k} на тестовом наборе:")

# Итерация по каждому тендеру в валидационном наборе
for tender_id, group in val_df.groupby("tender_id"):
    # Сортируем поставщиков внутри тендера по предсказанному скору (убывание)
    sorted_group = group.sort_values("pred_score", ascending=False)

    # Истинные метки релевантности (0 или 1) в отсортированном порядке
    true_relevance_sorted = sorted_group["y"].values
    
    pred_scores_sorted = sorted_group["pred_score"].values

    true_relevance_top_k = true_relevance_sorted[:k]
    pred_scores_top_k = pred_scores_sorted[:k]

    total_relevant_in_group = group["y"].sum()

    # --- Расчет Precision@k ---
    relevant_in_top_k = sum(true_relevance_top_k)
    precision_k = relevant_in_top_k / k if k > 0 else 0.0
    precisions_at_k.append(precision_k)

    # --- Расчет Recall@k ---
    recall_k = relevant_in_top_k / total_relevant_in_group if total_relevant_in_group > 0 else 0.0
    if total_relevant_in_group == 0 and relevant_in_top_k == 0:
         recall_k = 1.0
    recalls_at_k.append(recall_k)

    # --- Расчет F1-score@k ---
    if (precision_k + recall_k) > 0:
        f1_k = 2 * (precision_k * recall_k) / (precision_k + recall_k)
    else:
        f1_k = 0.0
    f1_scores_at_k.append(f1_k)

    # --- Расчет Average Precision (AP@k) для MAP@k ---
    ap_sum = 0.0
    hits = 0 
    if total_relevant_in_group > 0:
        for i in range(min(k, len(true_relevance_sorted))):
            if true_relevance_sorted[i] == 1:
                hits += 1
                ap_sum += hits / (i + 1)

        average_precision_k = ap_sum / total_relevant_in_group
    else:
        average_precision_k = 1.0 if relevant_in_top_k == 0 else 0.0
    average_precisions_at_k.append(average_precision_k)


mean_precision_at_k = np.mean(precisions_at_k) if precisions_at_k else 0.0
mean_recall_at_k = np.mean(recalls_at_k) if recalls_at_k else 0.0
mean_f1_at_k = np.mean(f1_scores_at_k) if f1_scores_at_k else 0.0
map_at_k = np.mean(average_precisions_at_k) if average_precisions_at_k else 0.0 

print(f"Средняя Precision@{k}:   {mean_precision_at_k:.2f}")
print(f"Средний Recall@{k}:      {mean_recall_at_k:.2f}")
print(f"Средний F1-score@{k}:    {mean_f1_at_k:.2f}")
print(f"MAP@{k} (Mean Avg Prec): {map_at_k:.2f}")


C:\Users\abuzu\AppData\Local\Temp\ipykernel_2324\3597537208.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['pred_score'] = val_preds_scores



Расчет метрик для K=5 на тестовом наборе:
Средняя Precision@5:   0.16
Средний Recall@5:      0.72
Средний F1-score@5:    0.26
MAP@5 (Mean Avg Prec): 0.52


In [30]:
# сохранение модели
model.save_model('rec_model_lightgbm.txt')